In [2]:
%%capture --no-stderr
!pip install -qU langchain-openai
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
my_secret_key = os.getenv("API_KEY")

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=my_secret_key)

In [74]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import CSVLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pandas as pd

In [12]:

df = pd.read_csv("Student Data Rows.csv")
df.head()

,Name,Field of study,Year of study,List of subjects,Preferred Learning Styles,Personal Objectives,Challenges,Extracurricular activities
0,Alice Smith,Computer Science,2,Data Structures;Algorithms;Operating Systems,Visual;Kinesthetic,Prepare for internship interviews,Time management,Robotics Club;Coding Competitions
1,John Doe,Biology,3,Genetics;Microbiology;Biochemistry,Auditory,Graduate with honors,Test anxiety,Debate Team;Volunteering at Animal Shelter
2,Mary Johnson,Mathematics,1,Calculus;Linear Algebra;Discrete Mathematics,Visual,Excel in all exams,Understanding abstract concepts,Math Club;Chess Club
3,Robert Brown,History,4,Modern History;Ancient Civilizations;World Wars,Auditory;Visual,Prepare for graduate school,Essay writing,Historical Reenactment Society;Student Council
4,Emily Davis,Mechanical Engineering,3,Thermodynamics;Fluid Mechanics;Material Science,Kinesthetic,Develop a senior project,Complex problem solving,Formula SAE;Basketball Team


In [15]:
loader = CSVLoader(file_path='Student Data Rows.csv')

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=my_secret_key))
retriever = vectorstore.as_retriever()


In [ ]:
"You are an assistant for creating personalized study plans for students."
"In the csv file you will find the following columns "
"Name: the name of the student "
"Field of study: which field is being studied by the student "
"Year of study: number of years student is studying"
"List of subjects: list of all the subjects of the student"
"Preferred Learning Styles: how the student wants to learn"
"Personal Objectives: what the student wants to achieve"
"Challenges: what challenges the student wants to overcome"
"Extracurricular activities: what the student is"

In [46]:
# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for creating personalized study plans for students."
    "In the csv file you will find the following columns "
    "Name: the name of the student "
    "Field of study: which field is being studied by the student "
    "Year of study: number of years student is studying"
    "List of subjects: list of all the subjects of the student"
    "Preferred Learning Styles: how the student wants to learn"
    "Personal Objectives: what the student wants to achieve"
    "Challenges: what challenges the student wants to overcome"
    "Extracurricular activities: what the student is"
    "if the student's name is not known only respond with 'I don't know the student'"
    "Write complete study plans for the student"
    "study plan should be a timeline from Monday to Friday for the student to complete there personal objectives"
    "Make sure to not overwork the student with the studies, also cater for the Extracurricular activities on Saturday and Sunday"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [53]:
print(df["Name"])

0        Alice Smith
1           John Doe
2       Mary Johnson
3       Robert Brown
4        Emily Davis
5     Michael Wilson
6    Sophia Martinez
7       James Taylor
8    Olivia Anderson
9       David Thomas
Name: Name, dtype: object


In [61]:
response = rag_chain.invoke({"input": "create a study plane for Alice Smith?"})
print(response["answer"])

**Study Plan for Alice Smith:**

**Monday:**
- Data Structures: Visual learning style
- Algorithms: Kinesthetic learning style

**Tuesday:**
- Operating Systems: Visual learning style

**Wednesday:**
- Data Structures: Kinesthetic learning style
- Algorithms: Visual learning style

**Thursday:**
- Operating Systems: Kinesthetic learning style

**Friday:**
- Review all subjects: Visual and Kinesthetic learning styles

**Saturday and Sunday:**
- Robotics Club: Enhance programming skills
- Coding Competitions: Apply knowledge and skills learned during the week

**Additional Notes:**
- Allocate specific time slots each day for studying to manage time effectively.
- Practice interview questions and coding problems regularly to prepare for internship interviews.
- Incorporate visual aids and hands-on activities in studying to cater to preferred learning styles.
- Stay consistent with study schedule and utilize breaks for relaxation to avoid burnout.
- Seek assistance from professors or peers

In [70]:
response = rag_chain.invoke({"input": "create a study plane for Mary Johnson?"})
print(response["answer"])

**Study Plan for Mary Johnson:**

**Monday:**
- Morning: Review Calculus notes using visual aids
- Afternoon: Practice solving Calculus problems visually
- Evening: Study Linear Algebra with visual tools

**Tuesday:**
- Morning: Focus on understanding abstract concepts in Linear Algebra
- Afternoon: Study Discrete Mathematics using visual examples
- Evening: Review all subjects studied during the day

**Wednesday:**
- Morning: Practice Calculus problems with visual aids
- Afternoon: Review Linear Algebra concepts visually
- Evening: Study Discrete Mathematics with visual tools

**Thursday:**
- Morning: Focus on understanding abstract concepts in Discrete Mathematics
- Afternoon: Practice solving problems visually in all subjects
- Evening: Review all subjects studied during the day

**Friday:**
- Morning: Take practice exams for all subjects
- Afternoon: Review any challenging topics with a visual approach
- Evening: Prepare study materials for the weekend

**Saturday and Sunday:**
- P

In [72]:
response = rag_chain.invoke({"input": "create a study plane for Michael Wilson?"})
print(response["answer"])

**Study Plan for Michael Wilson**

**Monday:**
- Morning: Review Cognitive Psychology (Visual)
- Afternoon: Study Developmental Psychology (Visual)
- Evening: Practice Public Speaking (Auditory)

**Tuesday:**
- Morning: Study Neuropsychology (Visual)
- Afternoon: Research assistant position preparation (Visual)
- Evening: Public speaking practice (Auditory)

**Wednesday:**
- Morning: Review Developmental Psychology (Visual)
- Afternoon: Research assistant position preparation (Visual)
- Evening: Public speaking practice (Auditory)

**Thursday:**
- Morning: Study Cognitive Psychology (Visual)
- Afternoon: Prepare for Research assistant position (Visual)
- Evening: Public speaking practice (Auditory)

**Friday:**
- Morning: Review Neuropsychology (Visual)
- Afternoon: Research assistant position preparation (Visual)
- Evening: Public speaking practice (Auditory)

**Saturday:**
- Morning: Yoga
- Afternoon: Drama Club

**Sunday:**
- Morning: Relaxation
- Afternoon: Drama Club

This study p

In [73]:
response = rag_chain.invoke({"input": "create a study plan for Raheem Tariq?"})
print(response["answer"])

I don't know the student
